# Without using keras

In [3]:
import numpy as np

# Activation function (sigmoid) and its derivative
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

# Define the training data for AND-NOT function
X_and_not = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y_and_not = np.array([[0], [0], [1], [0]])

# Define the training data for XOR function
X_xor = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y_xor = np.array([[0], [1], [1], [0]])

# Define the Multi-layer Perceptron class with one hidden layer
class MLP:
    def __init__(self, input_size, hidden_size, output_size):
        # Initialize weights for input to hidden layer
        self.weights_input_hidden = np.random.rand(input_size, hidden_size)
        
        # Initialize weights for hidden to output layer
        self.weights_hidden_output = np.random.rand(hidden_size, output_size)
        
        # Initialize biases for hidden layer
        self.bias_hidden = np.random.rand(1, hidden_size)
        
        # Initialize biases for output layer
        self.bias_output = np.random.rand(1, output_size)

    def forward(self, X):
        # Forward pass through the hidden layer
        self.hidden = sigmoid(np.dot(X, self.weights_input_hidden) + self.bias_hidden)
        
        # Forward pass through the output layer
        self.output = sigmoid(np.dot(self.hidden, self.weights_hidden_output) + self.bias_output)
        return self.output

    def backward(self, X, y, output):
        # Calculate the error for the output layer
        output_error = y - output
        output_delta = output_error * sigmoid_derivative(output)

        # Calculate the error for the hidden layer
        hidden_error = output_delta.dot(self.weights_hidden_output.T)
        hidden_delta = hidden_error * sigmoid_derivative(self.hidden)

        # Update weights and biases
        self.weights_hidden_output += self.hidden.T.dot(output_delta)
        self.weights_input_hidden += X.T.dot(hidden_delta)
        self.bias_hidden += np.sum(hidden_delta, axis=0, keepdims=True)
        self.bias_output += np.sum(output_delta, axis=0, keepdims=True)

    def train(self, X, y, epochs):
        for _ in range(epochs):
            output = self.forward(X)
            self.backward(X, y, output)

    def predict(self, X):
        return (self.forward(X) > 0.5).astype(int)

# Training the MLP for AND-NOT function
mlp_and_not = MLP(input_size=2, hidden_size=4, output_size=1)
mlp_and_not.train(X_and_not, y_and_not, epochs=5000)

# Training the MLP for XOR function
mlp_xor = MLP(input_size=2, hidden_size=4, output_size=1)
mlp_xor.train(X_xor, y_xor, epochs=5000)

# Print training results
print("AND-NOT Function:")
print("Predictions:\n", mlp_and_not.predict(X_and_not))

print("\nXOR Function:")
print("Predictions:\n", mlp_xor.predict(X_xor))




AND-NOT Function:
Predictions:
 [[0]
 [0]
 [1]
 [0]]

XOR Function:
Predictions:
 [[0]
 [1]
 [1]
 [0]]


In [4]:
# Optional

# Interactive testing function
def test_model(model):
    while True:
        try:
            # Get user input
            input_data = input("\nEnter two binary values (e.g., 0 1) or 'exit' to quit: ").strip()
            if input_data.lower() == 'exit':
                break
            
            # Convert input to numpy array
            input_values = np.array([list(map(int, input_data.split()))])
            
            # Predict using the model
            prediction = model.predict(input_values)
            
            # Print the prediction
            print(f"Input: {input_values[0]} -> Prediction: {prediction[0][0]}")
        except ValueError:
            print("Invalid input. Please enter two binary values (e.g., 0 1) or 'exit' to quit.")

# Test the models interactively
print("\nTesting AND-NOT Function:")
test_model(mlp_and_not)

print("\nTesting XOR Function:")
test_model(mlp_xor)



Testing AND-NOT Function:

Enter two binary values (e.g., 0 1) or 'exit' to quit: 0 1
Input: [0 1] -> Prediction: 0

Enter two binary values (e.g., 0 1) or 'exit' to quit: exit

Testing XOR Function:

Enter two binary values (e.g., 0 1) or 'exit' to quit: 0 1
Input: [0 1] -> Prediction: 1

Enter two binary values (e.g., 0 1) or 'exit' to quit: exit


# Using Keras

In [6]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

# AND-NOT function training data
and_not_inputs = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
and_not_labels = np.array([0, 0, 1, 0])

# XOR function training data
xor_inputs = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
xor_labels = np.array([0, 1, 1, 0])

# Function to create and train a model
def create_and_train_model(inputs, labels, epochs=1000):
    model = Sequential([
        Dense(2, input_dim=2, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(inputs, labels, epochs=epochs, verbose=0)
    return model

# Train the AND-NOT model
and_not_model = create_and_train_model(and_not_inputs, and_not_labels)

# Train the XOR model
xor_model = create_and_train_model(xor_inputs, xor_labels)

# Function to test the model with specific inputs
def test_model(model, inputs):
    predictions = model.predict(inputs)
    predictions = [round(pred[0]) for pred in predictions]
    return predictions

# Test the models with training data
print("AND-NOT Model Predictions on training data:")
print(test_model(and_not_model, and_not_inputs))

print("\nXOR Model Predictions on training data:")
print(test_model(xor_model, xor_inputs))

# Manually test specific input values
and_not_test_input = [[0, 1]]
xor_test_input = [[1, 0]]

print("\nAND-NOT Model Prediction for input [0, 1]:")
print(test_model(and_not_model, and_not_test_input))

print("\nXOR Model Prediction for input [1, 0]:")
print(test_model(xor_model, xor_test_input))


AND-NOT Model Predictions on training data:
1/1 [==============================] - 0s 262ms/step
[0, 0, 0, 0]

XOR Model Predictions on training data:
1/1 [==============================] - 0s 240ms/step
[0, 1, 1, 0]

AND-NOT Model Prediction for input [0, 1]:
1/1 [==============================] - 0s 237ms/step
[0]

XOR Model Prediction for input [1, 0]:
1/1 [==============================] - 0s 130ms/step
[1]
